# Named Entity Recognition

The Assignment has been divided into the following 3 parts:
  1. English
  2. Hindi
  3. Bangla

With each model having the following steps in coarse and fine grain setting:
1. Dataset Preparation
2. Feature Extraction
3. Model Definition
4. Training the Model
6. Model Evaluation

Fine grain setting has 36 * 3(I,O,B) tags  
Coarse grain setting has 6 * 3(I,O,B) tags


In [ ]:
%pip install pytorch-lightning -q -U
import torch
from sklearn.metrics import classification_report

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


## English 
Download the folder from "https://drive.google.com/drive/folders/1BZ_usq8IOBKxszRn0snlp-BqVB2Skuo1?usp=sharing" 

In [ ]:
#file locations in gdrive, kindly download the Multiconer folder from above link in the main gdrive page 
# train_path = '/content/drive/MyDrive/KGP_Courses/Deep_Learning/Assignment_2/Dataset/EN-English/en_train.conll'
# dev_path = '/content/drive/MyDrive/KGP_Courses/Deep_Learning/Assignment_2/Dataset/EN-English/en_dev.conll'
# test_path = '/content/drive/MyDrive/KGP_Courses/Deep_Learning/Assignment_2/Dataset/EN-English/en_test.conll'
train_path = '/content/drive/MyDrive/KGP_Courses/Deep_Learning/Assignment_2/Dataset/HI-Hindi/hi_train.conll'
dev_path = '/content/drive/MyDrive/KGP_Courses/Deep_Learning/Assignment_2/Dataset/HI-Hindi/hi_dev.conll'
test_path = '/content/drive/MyDrive/KGP_Courses/Deep_Learning/Assignment_2/Dataset/HI-Hindi/hi_test.conll'
# train_path = './multiconer2023/EN-English/en_train.conll'
# dev_path = './multiconer2023/EN-English/en_dev.conll'
# test_path = './multiconer2023/EN-English/en_test.conll'
# train_path = './multiconer2023/HI-Hindi/hi_train.conll'
# dev_path = './multiconer2023/HI-Hindi/hi_dev.conll'
# test_path = './multiconer2023/HI-Hindi/hi_test.conll'

#### Tag mapping logic

In [ ]:
#tag mapping to map fine to coarse grain setting
tag_dict = {
    'AnatomicalStructure':'Medical',
    'Disease':'Medical',
    'MedicalProcedure':'Medical',
    'Medication/Vaccine':'Medical',
    'Symptom':'Medical',
    'Clothing':'Product',
    'Drink':'Product',
    'Food':'Product',
    'OtherPROD':'Product',
    'Vehicle':'Product',
    'Artist':'Person',
    'Athlete':'Person',
    'Cleric':'Person',
    'OtherPER':'Person',
    'Politician':'Person',
    'Scientist':'Person',
    'SportsManager':'Person',
    'AerospaceManufacturer':'Group',
    'CarManufacturer':'Group',
    'MusicalGRP':'Group',
    'ORG':'Group',
    'OtherCORP':'Group',
    'PrivateCorp':'Group',
    'PublicCorp':'Group',
    'SportsGRP':'Group',
    'TechCORP':'Group',
    'ArtWork':'CreativeWorks',
    'MusicalWork':'CreativeWorks',
    'OtherCW':'CreativeWorks',
    'Software':'CreativeWorks',
    'VisualWork':'CreativeWorks',
    'WrittenWork':'CreativeWorks',
    'Facility':'Location',
    'HumanSettlement':'Location',
    'OtherLOC':'Location',
    'Station':'Location',
    }


### Parsing Function for both fine and grain setting

In [ ]:
def dataset_parser(file, setting):
  pairs = []
  with open(file, 'r', encoding='utf-8') as f:
    lines = f.readlines()
    list_of_dict_pairs = []
    for l in lines:
      l = l.strip()
      if l.startswith('#'):
        continue  # Ignore Empty and comment lines
      if l=='':
        if(len(list_of_dict_pairs)>0):
          pairs.append(list_of_dict_pairs)
          list_of_dict_pairs = []
        continue
      parts = l.split(' _ _ ')
      word = parts[0]
      tag = parts[-1]
      if setting == 'C': #coarse setting
        if len(tag)>=2:
          if tag[0]=='B' and tag[1]=='-':
            tag = "B-" + tag_dict[tag[2:]]
          elif tag[0]=='I' and tag[1]=='-':
            tag = "I-" + tag_dict[tag[2:]]
          elif tag[0]=='O' and tag[1]=='-':
            tag = "O-" + tag_dict[tag[2:]]
      list_of_dict_pairs.append({'word': word, 'tag': tag})
    if(len(list_of_dict_pairs)>0):
      pairs.append(list_of_dict_pairs)
  return pairs

In [ ]:
train_dataset_coarse = dataset_parser(train_path, 'C')
test_dataset_coarse = dataset_parser(test_path, 'C')
dev_dataset_coarse = dataset_parser(dev_path, 'C')
print(len(train_dataset_coarse))
print(len(test_dataset_coarse))
print(len(dev_dataset_coarse))
# print(train_dataset_coarse[0])
le = 0
for x in train_dataset_coarse:
    le += len(x)
print(le)
le = 0
for x in test_dataset_coarse:
    le += len(x)
print(le)
le = 0
for x in dev_dataset_coarse:
    le += len(x)
print(le)

9632
18399
514
159454
295289
8639


In [ ]:
SEQ_LEN = 30

In [ ]:
def preprocessor(data, word_dict, tag_dict):
    # Extract sentences and tags
    words = [[pair['word'].lower() for pair in sentence] for sentence in data]
    tags = [[pair['tag'] for pair in sentence] for sentence in data]

    for i in range(len(words)):
        while len(words[i]) < SEQ_LEN:
            words[i].append('<PAD>')
            tags[i].append('<PAD>')

        if len(words[i]) > SEQ_LEN:
            words[i] = words[i][:SEQ_LEN]
            tags[i] = tags[i][:SEQ_LEN]
    for temp_tags in tags:
        for tag in temp_tags:
            if tag not in tag_dict:
                tag_dict[tag] = len(tag_dict)
    
    for temp_words in words:
        for word in temp_words:
            if word not in word_dict:
                word_dict[word] = len(word_dict)

    # Convert words and tags to indices
    X = torch.tensor([[word_dict.get(word, 1) for word in sentence] for sentence in words], dtype=torch.int).type(torch.LongTensor)
    Y = torch.tensor([[tag_dict[tag] for tag in sentence] for sentence in tags], dtype=torch.int).type(torch.LongTensor)
    
    return X, Y

In [ ]:
word_dict_coarse = {"<PAD>": 0, "<UNK>": 1}
tag_dict_coarse = {"<PAD>": 0}

train_X_coarse, train_Y_coarse = preprocessor(train_dataset_coarse, word_dict_coarse, tag_dict_coarse)
dev_X_coarse, dev_Y_coarse = preprocessor(dev_dataset_coarse, word_dict_coarse, tag_dict_coarse)
test_X_coarse, test_Y_coarse = preprocessor(test_dataset_coarse, word_dict_coarse, tag_dict_coarse)

# Print the sizes of the datasets
print(f"Number of training examples: {len(train_X_coarse)}")
print(f"Number of validation examples: {len(dev_X_coarse)}")
print(f"Number of testing examples: {len(test_X_coarse)}")
print(f"Number of words in the vocabulary: {len(word_dict_coarse)}")
print(f"Number of tags in the vocabulary: {len(tag_dict_coarse)}")
print(train_X_coarse[0])

Number of training examples: 9632
Number of validation examples: 514
Number of testing examples: 18399
Number of words in the vocabulary: 30589
Number of tags in the vocabulary: 14
tensor([ 2,  3,  4,  5,  6,  7,  8,  9, 10,  0,  0,  0,  0,  0,  0,  0,  0,  0,
         0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0])


In [ ]:
train_dataset_fine = dataset_parser(train_path, 'F')
test_dataset_fine = dataset_parser(test_path, 'F')
dev_dataset_fine = dataset_parser(dev_path, 'F')


In [ ]:
word_dict_fine = {"<PAD>": 0, "<UNK>": 1}
tag_dict_fine = {"<PAD>": 0}

train_X_fine, train_Y_fine = preprocessor(train_dataset_fine, word_dict_fine, tag_dict_fine)
dev_X_fine, dev_Y_fine = preprocessor(dev_dataset_fine, word_dict_fine, tag_dict_fine)
test_X_fine, test_Y_fine = preprocessor(test_dataset_fine, word_dict_fine, tag_dict_fine)

# Print the sizes of the datasets
print(f"Number of training examples: {len(train_X_fine)}")
print(f"Number of validation examples: {len(dev_X_fine)}")
print(f"Number of testing examples: {len(test_X_fine)}")
print(f"Number of words in the vocabulary: {len(word_dict_fine)}")
print(f"Number of tags in the vocabulary: {len(tag_dict_fine)}")
print(train_X_fine[0])

Number of training examples: 9632
Number of validation examples: 514
Number of testing examples: 18399
Number of words in the vocabulary: 30589
Number of tags in the vocabulary: 68
tensor([ 2,  3,  4,  5,  6,  7,  8,  9, 10,  0,  0,  0,  0,  0,  0,  0,  0,  0,
         0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0])


#### Model Definition
Model is defined using PyTorch Lightning's LightningModule class, which allows to organize the training logic into separate methods, making the code easier to understand and maintain.

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
import pytorch_lightning as pl

class NER(pl.LightningModule):
    def __init__(self, vocab_size, tagset_size, embedding_dim, hidden_dim, num_layers=1, bidirectional=False):
        super().__init__()
        self.embedding = nn.Embedding(vocab_size, embedding_dim) #B * seq_len, B * seq_len * embedding_dim
        self.lstm = nn.LSTM(embedding_dim, hidden_dim, batch_first=True, num_layers=num_layers, bidirectional=bidirectional)
        #B * seq_len * embedding_dim -> B * seq_len * hidden_dim 
        
        if bidirectional:
            self.fc = nn.Linear(2*hidden_dim, tagset_size)
        else:
            self.fc = nn.Linear(hidden_dim, tagset_size)
        self.loss_fn = nn.CrossEntropyLoss()
    
    def forward(self, x):
        embeds = self.embedding(x)
        #pass the input to the model and calculate softmax to know probabilities
        lstm_out, _ = self.lstm(embeds)
        tag_space = self.fc(lstm_out)
        tag_scores = nn.functional.log_softmax(tag_space, dim=2)
        return tag_scores
    
    def training_step(self, batch, batch_idx):
        x, y = batch
        y_hat = self.forward(x)
        loss = self.loss_fn(y_hat.view(-1, y_hat.shape[-1]), y.view(-1))
        self.log('train_loss', loss)
        return loss
    
    def validation_step(self, batch, batch_idx):
        x, y = batch
        y_hat = self.forward(x)
        loss = self.loss_fn(y_hat.view(-1, y_hat.shape[-1]), y.view(-1))
        self.log('val_loss', loss)
        return loss

    def test_step(self, batch, batch_idx):
        x, y = batch
        y_hat = self.forward(x)
        loss = self.loss_fn(y_hat.view(-1, y_hat.shape[-1]), y.view(-1))
        self.log('test_loss', loss)
        return loss
    
    def configure_optimizers(self):
        optimizer = optim.Adam(self.parameters())
        return optimizer

The class defines constructor, forward propagation logic, validation logic, test logic and optimization configuration to define model.

 `pl.LightningModule` is inherited to get access to PyTorch Lightning's training loop.

#### Training the Model
 PyTorch Lightning's Trainer class is used to train our model to take care of setting up the training loop, optimizing the model, and handling GPU acceleration.

In [ ]:
from torch.utils.data import DataLoader, TensorDataset
from pytorch_lightning.callbacks.early_stopping import EarlyStopping

EMBEDDING_DIM = 100
HIDDEN_DIM    = 100
NUM_EPOCHS    = 10 
BATCH_SIZE    = 10

train_dataset_coarse = TensorDataset(train_X_coarse, train_Y_coarse)
train_loader_coarse = DataLoader(train_dataset_coarse, batch_size=BATCH_SIZE, shuffle=True)

val_dataset_coarse = TensorDataset(dev_X_coarse, dev_Y_coarse)
val_loader_coarse = DataLoader(val_dataset_coarse, batch_size=BATCH_SIZE)

test_dataset_coarse = TensorDataset(test_X_coarse, test_Y_coarse)
test_loader_coarse = DataLoader(test_dataset_coarse, batch_size=BATCH_SIZE)

train_dataset_fine = TensorDataset(train_X_fine, train_Y_fine)
train_loader_fine = DataLoader(train_dataset_fine, batch_size=BATCH_SIZE, shuffle=True)

val_dataset_fine = TensorDataset(dev_X_fine, dev_Y_fine)
val_loader_fine = DataLoader(val_dataset_fine, batch_size=BATCH_SIZE)

test_dataset_fine = TensorDataset(test_X_fine, test_Y_fine)
test_loader_fine = DataLoader(test_dataset_fine, batch_size=BATCH_SIZE)

#### Uncomment the below lines of code to train the coarse model

In [ ]:
model = NER(vocab_size=len(word_dict_coarse), tagset_size=len(tag_dict_coarse), embedding_dim=EMBEDDING_DIM, hidden_dim=HIDDEN_DIM, bidirectional=True)
early_stopping = EarlyStopping(monitor="val_loss", patience=4, mode="min")
trainer = pl.Trainer(max_epochs=NUM_EPOCHS, callbacks=[early_stopping])
trainer.fit(model, train_dataloaders=train_loader_coarse, val_dataloaders=val_loader_coarse)
trainer.test(dataloaders=test_loader_coarse)

INFO:pytorch_lightning.utilities.rank_zero:GPU available: True (cuda), used: True
INFO:pytorch_lightning.utilities.rank_zero:TPU available: False, using: 0 TPU cores
INFO:pytorch_lightning.utilities.rank_zero:IPU available: False, using: 0 IPUs
INFO:pytorch_lightning.utilities.rank_zero:HPU available: False, using: 0 HPUs
INFO:pytorch_lightning.accelerators.cuda:LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
INFO:pytorch_lightning.callbacks.model_summary:
  | Name      | Type             | Params
-----------------------------------------------
0 | embedding | Embedding        | 3.1 M 
1 | lstm      | LSTM             | 161 K 
2 | fc        | Linear           | 2.8 K 
3 | loss_fn   | CrossEntropyLoss | 0     
-----------------------------------------------
3.2 M     Trainable params
0         Non-trainable params
3.2 M     Total params
12.893    Total estimated model params size (MB)


Sanity Checking: 0it [00:00, ?it/s]

Training: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

...

Validation: 0it [00:00, ?it/s]

/usr/local/lib/python3.9/dist-packages/pytorch_lightning/trainer/connectors/checkpoint_connector.py:148: UserWarning: `.test(ckpt_path=None)` was called without a model. The best model of the previous `fit` call will be used. You can pass `.test(ckpt_path='best')` to use the best model or `.test(ckpt_path='last')` to use the last model. If you pass a value, this warning will be silenced.
  rank_zero_warn(
INFO:pytorch_lightning.utilities.rank_zero:Restoring states from the checkpoint path at /content/lightning_logs/version_4/checkpoints/epoch=7-step=7712.ckpt
INFO:pytorch_lightning.accelerators.cuda:LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
INFO:pytorch_lightning.utilities.rank_zero:Loaded model weights from the checkpoint at /content/lightning_logs/version_4/checkpoints/epoch=7-step=7712.ckpt


Testing: 0it [00:00, ?it/s]

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃        Test metric        ┃       DataLoader 0        ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│         test_loss         │    0.2079976350069046     │
└───────────────────────────┴───────────────────────────┘

[{'test_loss': 0.2079976350069046}]

In [ ]:
torch.save(model.state_dict(), 'drive/MyDrive/KGP_Courses/Deep_Learning/Assignment_2/Model_Parameters/HI-Hindi/en_coarse.pt')

In [ ]:
drive.mount('/content/drive')
path = '/content/drive/MyDrive/KGP_Courses/Deep_Learning/Assignment_2/Model_Parameters/HI-Hindi/en_coarse.pt'
#download the en_coarse.pt from 

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
en_coarse = NER(vocab_size=len(word_dict_coarse), tagset_size=len(tag_dict_coarse), embedding_dim=EMBEDDING_DIM, hidden_dim=HIDDEN_DIM, bidirectional=True)
en_coarse.load_state_dict(torch.load(path))

<All keys matched successfully>

In [ ]:
en_coarse

NER(
  (embedding): Embedding(30589, 100)
  (lstm): LSTM(100, 100, batch_first=True, bidirectional=True)
  (fc): Linear(in_features=200, out_features=14, bias=True)
  (loss_fn): CrossEntropyLoss()
)

In [ ]:
def accuracy(tag_to_idx_coarse,test_dataset_coarse,model):
  # define idx_to_tag
  idx_to_tag = {idx: tag for tag, idx in tag_to_idx_coarse.items()}

  # define device
  device = torch.device('cpu')

  # Create a dataloader for the test set
  test_loader = DataLoader(test_dataset_coarse, batch_size=BATCH_SIZE)

  # Set the model to evaluation mode
  model.eval()

  y_true = []
  y_pred = []

  with torch.no_grad():
      for x, y in test_loader:
          # Move the data to the device
          x = x.to(device)
          y = y.to(device)

          # Forward pass
          y_hat = model(x)

          # Compute the predicted tags
          y_pred += [idx_to_tag[i] for i in y_hat.argmax(-1).cpu().numpy().flatten().tolist()]

          # Compute the true tags
          y_true += [idx_to_tag[i] for i in y.cpu().numpy().flatten().tolist()]

  print(classification_report(y_true, y_pred))

In [ ]:
accuracy(tag_dict_coarse,test_dataset_coarse,en_coarse)

                 precision    recall  f1-score   support

          <PAD>       1.00      1.00      1.00    258203
B-CreativeWorks       0.58      0.55      0.57      2802
        B-Group       0.76      0.75      0.75      3893
     B-Location       0.80      0.63      0.70      7171
      B-Medical       0.66      0.68      0.67      1973
       B-Person       0.69      0.65      0.67      5732
      B-Product       0.54      0.57      0.56      1608
I-CreativeWorks       0.73      0.49      0.59      3908
        I-Group       0.85      0.80      0.82      5567
     I-Location       0.77      0.69      0.73      3251
      I-Medical       0.71      0.65      0.68       800
       I-Person       0.74      0.66      0.70      6532
      I-Product       0.64      0.54      0.58       755
              O       0.96      0.98      0.97    249775

       accuracy                           0.96    551970
      macro avg       0.75      0.69      0.71    551970
   weighted avg       0.96   

#### Uncomment the below lines of code to train the fine model

In [ ]:
model = NER(vocab_size=len(word_dict_fine), tagset_size=len(tag_dict_fine), embedding_dim=EMBEDDING_DIM, hidden_dim=HIDDEN_DIM, bidirectional=True)
early_stopping = EarlyStopping(monitor="val_loss", patience=4, mode="min")
trainer = pl.Trainer(max_epochs=NUM_EPOCHS, callbacks=[early_stopping])
trainer.fit(model, train_dataloaders=train_loader_fine, val_dataloaders=val_loader_fine)

trainer.test(dataloaders=test_loader_fine)

INFO:pytorch_lightning.utilities.rank_zero:GPU available: True (cuda), used: True
INFO:pytorch_lightning.utilities.rank_zero:TPU available: False, using: 0 TPU cores
INFO:pytorch_lightning.utilities.rank_zero:IPU available: False, using: 0 IPUs
INFO:pytorch_lightning.utilities.rank_zero:HPU available: False, using: 0 HPUs
INFO:pytorch_lightning.accelerators.cuda:LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
INFO:pytorch_lightning.callbacks.model_summary:
  | Name      | Type             | Params
-----------------------------------------------
0 | embedding | Embedding        | 3.1 M 
1 | lstm      | LSTM             | 161 K 
2 | fc        | Linear           | 13.7 K
3 | loss_fn   | CrossEntropyLoss | 0     
-----------------------------------------------
3.2 M     Trainable params
0         Non-trainable params
3.2 M     Total params
12.937    Total estimated model params size (MB)


Sanity Checking: 0it [00:00, ?it/s]

Training: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

...

Validation: 0it [00:00, ?it/s]

/usr/local/lib/python3.9/dist-packages/pytorch_lightning/trainer/connectors/checkpoint_connector.py:148: UserWarning: `.test(ckpt_path=None)` was called without a model. The best model of the previous `fit` call will be used. You can pass `.test(ckpt_path='best')` to use the best model or `.test(ckpt_path='last')` to use the last model. If you pass a value, this warning will be silenced.
  rank_zero_warn(
INFO:pytorch_lightning.utilities.rank_zero:Restoring states from the checkpoint path at /content/lightning_logs/version_5/checkpoints/epoch=8-step=8676.ckpt
INFO:pytorch_lightning.accelerators.cuda:LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
INFO:pytorch_lightning.utilities.rank_zero:Loaded model weights from the checkpoint at /content/lightning_logs/version_5/checkpoints/epoch=8-step=8676.ckpt


Testing: 0it [00:00, ?it/s]

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃        Test metric        ┃       DataLoader 0        ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│         test_loss         │    0.28593671321868896    │
└───────────────────────────┴───────────────────────────┘

[{'test_loss': 0.28593671321868896}]

In [ ]:
torch.save(model.state_dict(), 'drive/MyDrive/KGP_Courses/Deep_Learning/Assignment_2/Model_Parameters/HI-Hindi/en_fine.pt')

In [ ]:
path = '/content/drive/MyDrive/KGP_Courses/Deep_Learning/Assignment_2/Model_Parameters/HI-Hindi/en_fine.pt'
#download the en_coarse.pt from 

en_fine = NER(vocab_size=len(word_dict_fine), tagset_size=len(tag_dict_fine), embedding_dim=EMBEDDING_DIM, hidden_dim=HIDDEN_DIM, bidirectional=True)
en_fine.load_state_dict(torch.load(path))

<All keys matched successfully>

In [ ]:
en_fine

NER(
  (embedding): Embedding(30589, 100)
  (lstm): LSTM(100, 100, batch_first=True, bidirectional=True)
  (fc): Linear(in_features=200, out_features=68, bias=True)
  (loss_fn): CrossEntropyLoss()
)

In [ ]:
accuracy(tag_dict_fine,test_dataset_fine,en_fine)

                         precision    recall  f1-score   support

                  <PAD>       1.00      1.00      1.00    258203
B-AerospaceManufacturer       0.44      0.05      0.09        85
  B-AnatomicalStructure       0.67      0.69      0.68       489
              B-ArtWork       0.17      0.00      0.00       426
               B-Artist       0.51      0.38      0.44      1852
              B-Athlete       0.70      0.55      0.61      1171
      B-CarManufacturer       0.70      0.89      0.79       146
               B-Cleric       0.57      0.80      0.66       188
             B-Clothing       0.59      0.81      0.68        77
              B-Disease       0.72      0.62      0.67       633
                B-Drink       0.57      0.74      0.64       135
             B-Facility       0.41      0.47      0.43       859
                 B-Food       0.59      0.55      0.57       428
      B-HumanSettlement       0.77      0.64      0.70      5825
     B-MedicalProcedure 

#### Use the pretrained model

In [ ]:
def predict(tag_dict,word_dict,line,model):
  # define idx_to_tag
  idx_to_tag = {idx: tag for tag, idx in tag_dict.items()}

  # define device
  device = torch.device('cpu')

  # Set the model to evaluation mode
  model.eval()

  y_pred = []
  x = []

  sent = line.split()
  if(len(sent)>30):
    i=0  
    for word in sent and i!=30:
      if word in word_dict:
        x.append(word_dict[word])
      else:
        x.append(1)
      i=i+1
  else:
    i=0  
    for word in sent:
      if word in word_dict:
        x.append(word_dict[word])
      else:
        x.append(1)
    while i != 30-len(sent):
      x.append(0)
      i=i+1

    x = torch.tensor([x])
    with torch.no_grad():
       # Move the data to the device
        x = x.to(device)
        
        # Forward pass
        y_hat = model(x)

        # Compute the predicted tags
        y_pred += [idx_to_tag[i] for i in y_hat.argmax(-1).cpu().numpy().flatten().tolist()]
        # print(y_pred)
  return y_pred

In [ ]:
y_pred = predict(tag_dict_coarse,word_dict_coarse, "robert gottschalk 1939 academy award winner and founder of pranavision ",en_coarse)
y_pred

['B-Product',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 '<PAD>',
 '<PAD>',
 '<PAD>',
 '<PAD>',
 '<PAD>',
 '<PAD>',
 '<PAD>',
 '<PAD>',
 '<PAD>',
 '<PAD>',
 '<PAD>',
 '<PAD>',
 '<PAD>',
 '<PAD>',
 '<PAD>',
 '<PAD>',
 '<PAD>',
 '<PAD>',
 '<PAD>',
 '<PAD>']

In [ ]:
y_pred = predict(tag_dict_fine,word_dict_fine, "robert gottschalk 1939 academy award winner and founder of pranavision ",en_fine)
y_pred

['B-OtherPER',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'I-Artist',
 'O',
 '<PAD>',
 '<PAD>',
 '<PAD>',
 '<PAD>',
 '<PAD>',
 '<PAD>',
 '<PAD>',
 '<PAD>',
 '<PAD>',
 '<PAD>',
 '<PAD>',
 '<PAD>',
 '<PAD>',
 '<PAD>',
 '<PAD>',
 '<PAD>',
 '<PAD>',
 '<PAD>',
 '<PAD>',
 '<PAD>']